## Aggregations

In [14]:
import sqlite3
import pandas as pd

### Connection to sqlite3

In [15]:
conn = sqlite3.connect("../ex00/data/checking-logs.sqlite")

### Get the schema

In [16]:
schema_test = pd.read_sql("PRAGMA table_info(test);", conn)
schema_test

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


### First 10 rows of the table

In [17]:
df_test = pd.read_sql("SELECT * FROM test LIMIT 10;", conn)
df_test

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


### Min delta

In [ ]:
query_min = """
SELECT test.uid,
    MIN((UNIXEPOCH(test.first_commit_ts) - deadlines.deadlines) / 3600) AS min_delta_hours
FROM test
JOIN deadlines ON deadlines.labs = test.labname
WHERE test.labname != 'project1'
ORDER BY min_delta_hours ASC;
"""

df_min = pd.read_sql(query_min, conn)
df_min

,uid,min_delta_hours
0,user_30,-202


### Max delta

In [19]:
query_max = """
SELECT test.uid,
    MAX((UNIXEPOCH(test.first_commit_ts) - deadlines.deadlines) / 3600) AS max_delta_hours
FROM test
JOIN deadlines ON deadlines.labs = test.labname
WHERE test.labname != 'project1'
ORDER BY max_delta_hours DESC;
"""
df_max = pd.read_sql(query_max, conn)
df_max


,uid,max_delta_hours
0,user_25,-2


### Avarage delta

In [20]:
query_avg = """
SELECT 
    AVG((UNIXEPOCH(test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_delta_hours
FROM test
JOIN deadlines ON deadlines.labs = test.labname
WHERE test.labname != 'project1'
ORDER BY avg_delta_hours ASC;
"""
df_avg = pd.read_sql(query_avg, conn)
df_avg

,avg_delta_hours
0,-89.125


### Hypothesis(gipoteza) test: newsfeed ko‘rishlar soni bilan bog‘liqlik

In [21]:
query_views_diff = """
SELECT  
    test.uid,
    AVG((UNIXEPOCH(test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_diff,
    COUNT(pviews.datetime) as pageviews
FROM test
JOIN deadlines ON deadlines.labs = test.labname
LEFT JOIN pageviews pviews ON test.uid = pviews.uid
WHERE test.labname != 'project1'
GROUP BY test.uid;
"""
views_diff = pd.read_sql(query_views_diff, conn)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,140
1,user_10,-74.800000,445
2,user_14,-159.000000,429
3,user_17,-61.600000,235
4,user_18,-5.666667,9
5,user_19,-98.750000,64
6,user_21,-95.500000,40
7,user_25,-92.600000,895
8,user_28,-86.400000,745
9,user_3,-105.400000,1585


### Correlation coefficient

In [22]:
correlation = views_diff.drop(columns=['uid']).corr()
correlation

,avg_diff,pageviews
avg_diff,1.000000,-0.185834
pageviews,-0.185834,1.000000


### Connection closed

In [23]:
conn.close()